#  Action Rules Discovery - Titanic Case Study

This notebook presents a new implemetation of several action rule mining algorithms. Action rule mining was first introduced in:
    
    Ras, Zbigniew W., and Alicja Wieczorkowska. "Action-rules: How to increase profit of a company." European Conference on Principles of Data Mining and Knowledge Discovery. Springer, Berlin, Heidelberg, 2000.

Action rule mining is an extension of the widely used task of learning classification rules. In addition to information expressed in a standard classification rules, an action rule:

* Suggests a coarse of action, which could change the class assigned to instances covered by the rule.
* The course of action is realistic in that it involves changing of values only of influenceable (actionable) attributes


## Example

For illustration, we will use the well-known  Titanic dataset. 

---
** STANDARD CLASSIFICATION RULE**:


    IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=YES  
    TODO: Add confidence + support + explain below
---

This rule predicts chances of survival for a particular person. An action rule is an extended classification rule, which also says what that person should have done differently to increase the chance of survival. 
In the Titanic dataset, the passenger cannot influcence his or her age or gender, but can influence the passenger class and the boarding place - these become the actionable attributes.

---
** ACTION RULE**:

    Women who traveled from Southampton in the third class would increase their chance for survival if they would change their boarding place to Cherbourg and paid extra money for the first class.

---

## Required inputs

* Tabular dataset
* Specification of target class
* List of actionable attributes


## Installation of the package
        
    
    pip install -i https://test.pypi.org/simple/ actionrules-lukassykora
        
For mining candidate rules, the actionrules package uses the very fast pyFIM package.
This package is not availabe in the pypi index, but it can be easily installed following these instructions:  http://www.borgelt.net/pyfim.html.


## Action rule mining Python interface

As the first step, the module needs to be imported:

In [16]:
import pandas as pd
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

### Instantiate model object and load data

All columns in the input dataset must be nominal. Note: numerical variables can be converted to nominal with  discretization (binning). 

In [13]:
dataFrame = pd.read_csv("data/titanic.csv", sep=";")
dataFrame.head()

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
0,1,<16.13336;32.10002),S,very high,1.0,female,1.0
1,2,<0.1667;16.13336),S,very high,1.0,male,1.0
2,3,<0.1667;16.13336),S,very high,1.0,female,0.0
3,4,<16.13336;32.10002),S,very high,1.0,male,0.0
4,5,<16.13336;32.10002),S,very high,1.0,female,0.0


In [14]:
dataFrame[dataFrame.Sex=="female"][dataFrame.Embarked == "S"][dataFrame.Survived == 1.0]

/home/tomas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,ID,Age,Embarked,Fare,Pclass,Sex,Survived
0,1,<16.13336;32.10002),S,very high,1.0,female,1.0
6,7,<48.06668;64.03334),S,very high,1.0,female,1.0
8,9,<48.06668;64.03334),S,very high,1.0,female,1.0
13,14,<16.13336;32.10002),S,very high,1.0,female,1.0
21,22,<32.10002;48.06668),S,very high,1.0,female,1.0
24,25,<16.13336;32.10002),S,very high,1.0,female,1.0
28,29,<32.10002;48.06668),S,very high,1.0,female,1.0
32,33,<16.13336;32.10002),S,very high,1.0,female,1.0
33,34,<48.06668;64.03334),S,higher,1.0,female,1.0
36,37,<16.13336;32.10002),S,very high,1.0,female,1.0


### Configuration options

For mining from the input dataset, the **fit**  method can be used. Internally,  ActionRulesDiscovery uses the PyFIM library for classification rules discovery.  The fit function has a number of options and parameters:

In [17]:
help(ActionRulesDiscovery.fit)

Help on function fit in module actionrules.actionRulesDiscovery.actionRulesDiscovery:

fit(self, stable_antecedents:List[str], flexible_antecedents:List[str], consequent:str, conf:float, supp:float, desired_classes:List[str]=None, desired_changes:List[list]=None, is_nan:bool=False, is_reduction:bool=True, min_stable_antecedents:int=1, min_flexible_antecedents:int=1, max_stable_antecedents:int=5, max_flexible_antecedents:int=5)
    Get action rules.
    Define antecedent and consequent.
    - stable_antecedents - List of column names.
    - flexible_antecedents - List of column names.
    - consequent - Name of the consequent column.
    Confidence and support.
    - conf - Value in % for minimal confidence in classification rules.
             For example, 60.
    - supp - Value in % for minimal support of classification rules.
             For example, 5.
    Desired classes or desired changes must be entered.
    - desired_classes - List of decision states. For example, ["1"].
      

## Fit model to training data


In our example, we will focus on finding rules that provide actionable advice to passengers who have not survived.


In [18]:
actionRulesDiscovery = ActionRulesDiscovery()
actionRulesDiscovery.load_pandas(dataFrame)
actionRulesDiscovery.fit(stable_antecedents = ["Age", "Sex"],
            flexible_antecedents = ["Embarked", "Fare", "Pclass"],
            consequent = "Survived",
            conf=55,
            supp=3,
            desired_classes = ["1.0"],
)

#### Setting the target

The target attribute is set as follows:

    consequent = "Survived"
    
In our example, we will focus on finding rules that provide actionable advise to passengers who have not survived (Survived=1.0).

    desired_classes = ["1.0"]

We want to find rules which will provide advice on actions, which will result in change of the classification to survived (which is coded as 1.0 in data). Since the only other value of the class variable is 0.0 (Not survived), the learning will focus on finding rules that change the classification from `not survived` to `survived`.

#### Setting minimum support
We intend to discover action rules from classification rules that have a minimum support of 3%.

    supp=3


#### What is a support of a rule?

Consider classification rule `IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=YES`. Our complete dataset covers 1310 passengers. Out of these, there are 198 passangers who match the complete rule (surviving females boarded in Southampton). The support of this rule is thus 198/1310= 15.1%.

#### Setting minimum confidence
We want to discover action rules from classification rules that have a minimum confidence of 55%.

    conf=55


####  What is a confidence of a rule?

Consider classification rule `IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=NO`.

Let us assume that in the data, there are 291 passangers who match the antecedent of this rule (boarded in  Southampton and are females). Out of this group, 198 passengers have survived. The confidence of the rule is thus  198/291 = 68%.

####  Defining stable and actionable attributes
For all attributes in the passed dataset, we need to define whether they are stable or actionable. 
"Age" and "Sex" are designated as stable, because they cannot be changed -  an action rule proposing change in sex or age would be useless.

    stable_antecedents = ["Age", "Sex"]    

As flexible (actionable) attributes we use "Embarked", "Fare", "Pclass". Flexible attributes allow changing of their state (an action can be taken). 

    flexible_antecedents = ["Embarked", "Fare", "Pclass"]   



#### Number of discovered action rules

In [19]:
len(actionRulesDiscovery.get_pretty_action_rules())

13

## Interpreting the results

### Obtaining all discovered action rules

In [20]:
for rule in actionRulesDiscovery.get_action_rules_representation():
    print(rule)
    print(" ")

r = [(Sex: female) ∧ (Embarked: S → C)  ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.05267175572519084 and confidence: 0.5876187356698329
 
r = [(Age: <16.13336;32.10002)) ∧ (Embarked: S → C) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04198473282442748 and confidence: 0.40990990990991
 
r = [(Age: <16.13336;32.10002)) ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04732824427480916 and confidence: 0.5261127825349071
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: avg → very high) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.044274809160305344 and confidence: 0.43256997455470736
 
r = [(Age: <16.13336;32.10002)) ∧ (Pclass: 2.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04732824427480916 and confidence: 0.42040850078557845
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: avg → very high)  ∧ (Pclass: 2.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.03969465648854962 and confidence: 0.46254681647940077
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: very low → ver

## Interpretation of a selected action rule 

We will analyze the fist rule on the output:

In [22]:
print(actionRulesDiscovery.get_action_rules_representation()[0])

r = [(Sex: female) ∧ (Embarked: S → C)  ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.05267175572519084 and confidence: 0.5876187356698329



#### Obtaining details of a selected rule
We can obtain details of a selected rule using:

In [24]:
actionRulesDiscovery.get_action_rules()[0]

[[[['Sex', ('female',)]],
  [['Embarked', ('S', 'C')], ['Pclass', ('3.0', '1.0')]],
  ['Survived', ['0.0', '1.0']]],
 [0.059541984732824425, 0.05267175572519084, 0.05267175572519084],
 [0.6046511627906976, 0.971830985915493, 0.5876187356698329]]

The output of `actionRulesDiscovery.get_action_rules()` is a list of action rules. Each action rule is a list where the first part is an action rule itself, and the second part is a tuple of (support before, support after, action rule support) and (confidence before, confidence after, action rule confidence).

#### Text representation of selected action rule - auto generated


A selected action rule can also be represented in a self-explanatory, yet a bit more verbose, textual notation:

In [30]:
actionRulesDiscovery.get_pretty_action_rules()[0]

"If attribute 'Sex' is 'female', attribute 'Embarked' value 'S' is changed to 'C', attribute 'Pclass' value '3.0' is changed to '1.0', then 'Survived' value '0.0' is changed to '1.0' with support: 0.05267175572519084 and confidence: 0.5876187356698329."

#### Text representation of selected action rule - manual

A selected action rule can also be represented in a friendlier, yet a bit more verbose, textual notation:

**Women** who traveled from **Southampton** in the **third class** would increase their chance for survival if they would change their boarding place to **Cherbourg** and paid extra money for **first class**.

This notation cannot currently be generated algorithmically.

### Parts of  action rule



#### Antecedent of the rule 
    [(Sex: female) ∧ (Embarked: S → C) ∧ (Pclass: 3.0 → 1.0) ] 

#### Consequent of the rule
     [Survived: 0.0 → 1.0] 

#### Decomposing action rule to Rule Before and Rule After
A single action rule can actually **be decomposed to two rules**, one representing the state *before* and the second one *after* the intervention.

* **Rule before**: r1 = [(Sex: female) ∧ (Embarked: S)  ∧ (Pclass: 3.0) ] ⇒ [Survived: 0.0] 

* **Rule after**:  r2 = [(Sex: female) ∧ (Embarked: C)  ∧ (Pclass: 1.0) ] ⇒ [Survived: 1.0] 


#### Individual conditions

     Sex: female

This rule applies to passangers, who are females,  


    Embarked: S
who boarded in Southampton, and 

    Pclass: 3.0
who travelled in the third class.

If these passengers would

    Embarked: S → C
change the boarding place to Cherbourg and at the same time
    
    Pclass: 3.0 → 1.0
change the passanger class from third class to first class

    [Survived: 0.0 → 1.0]
it will become more likely that they would survive,

    support: 0.05267175572519084 and confidence: 0.5876187356698329
The rule will will apply to 5% of passangers.
The recommended action will change the outcome with probability of 59%.
The way these measures are computed is described in the following.
    

#### Additional measures of quality of the action rule
The concise representation of an action rule as shown above,  lists only the Confidence of Action Rule and Support of Action Rule. Detailed representation of the rule, obtained with `actionRulesDiscovery.get_action_rules()`, returns:
    
    Support Before 0.06
    Support After 0.05
    Support of Action Rule: 0.05
    Confidence Before: 0.60
    Confidence After: 0.97
    Confidence of Action Rule: 0.59


* ** Support Before**: Support of the Rule before, *sup(r1) = number of instances matching all conditions in the antecedent and consequent of r1*

* ** Support After**: Support of the Rule after, *sup(r2)  number of instances matching all conditions in the antecedent and consequent of r2*

* ** Support of Action Rule** Support of the action rule r is computed from rules r1 (Before) and r2 (After) as

`sup(r) = min(sup(r1), sup(r2))`. 

It can be interpreted as  the percentage of passengers the rule applies to.

* ** Confidence Before: ** Confidence of the Rule before, *conf(r1) =  number of instances matching all conditions in the antecedent and consequent of r1  divided by  number of instances matching all conditions in the antecedent of r1*

* ** Confidence After: ** Confidence of the Rule after, *conf(r1) =  number of instances matching all conditions in the antecedent and consequent of r2  divided by  number of instances matching all conditions in the antecedent of r2*

* ** Confidence of Action Rule: ** Confidence of the action rule r is computed from classification rules r1 (Before) and r2 (After) as 

`conf(r) = conf(r1) ∗ conf(r2)`. 

It can be interpreted as the percentage of female passengers, for whom the survival outlook would change from "not survive" to "survive"  if they would have followed the recommendation of the action rule: change boarding place from Southampton to Cherbourg, and third-class to the first-class.

## How it works

### First action rule - before
The following code displays data that show instances in data that match the conditions of the "before" part of the action rule.

- Yellow background - stable attributes

- Orange background - flexible attributes

- Red text - Target value, undesired state

- Green text - Target attribute, desired state

Note that the color highlighting does not work on Github.

You can notice that "Not survived" is the prevalent value.

In [31]:
actionRulesDiscovery.get_source_data_for_ar(0, True)

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
603,604,<32.10002;48.06668),S,avg,3.0,female,1.0
604,605,<0.1667;16.13336),S,very low,3.0,female,1.0
610,611,<32.10002;48.06668),S,lower,3.0,female,0.0
612,613,<16.13336;32.10002),S,lower,3.0,female,1.0
621,622,<16.13336;32.10002),S,lower,3.0,female,1.0
623,624,<0.1667;16.13336),S,higher,3.0,female,0.0
624,625,<0.1667;16.13336),S,higher,3.0,female,0.0
625,626,<16.13336;32.10002),S,lower,3.0,female,1.0
626,627,<32.10002;48.06668),S,very low,3.0,female,0.0
627,628,<0.1667;16.13336),S,higher,3.0,female,0.0


### First action rule - after
The following code displays data that show instances in data that match the conditions of the "after" part of the action rule.

- Yellow background - stable attributes

- Orange background - flexible attributes

- Red text - Target attribute, undesired state

- Green text - Target attribute, desired state

Notice that nearly all instances have the outcome Survived=1.0, which indicates survival.

In [32]:
actionRulesDiscovery.get_source_data_for_ar(0, False)

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
11,12,<16.13336;32.10002),C,very high,1.0,female,1.0
12,13,<16.13336;32.10002),C,very high,1.0,female,1.0
17,18,<48.06668;64.03334),C,very high,1.0,female,1.0
18,19,<16.13336;32.10002),C,very high,1.0,female,1.0
23,24,<32.10002;48.06668),C,very high,1.0,female,1.0
27,28,<16.13336;32.10002),C,very high,1.0,female,1.0
35,36,<32.10002;48.06668),C,very high,1.0,female,1.0
41,42,<32.10002;48.06668),C,higher,1.0,female,1.0
43,44,<48.06668;64.03334),C,very high,1.0,female,1.0
44,45,<32.10002;48.06668),C,very high,1.0,female,1.0


### Prediction
In the context of action rule mining, prediction means finding action rules that matches a given instance. Based on these rules, recommendations are generated.

Consider the following test instance:

In [33]:
new_data = [['<32.10002;48.06668)','female', 'S', 'very high' ,'3.0']] 
df_new_data = pd.DataFrame(new_data, columns = ["Age", "Sex", "Embarked", "Fare", "Pclass"]) 

The prediction is invoked as follows: 

In [34]:
actionRulesDiscovery.predict(df_new_data)

,Age,Embarked,Embarked-recommended,Fare,Pclass,Pclass-recommended,Sex,action rule,action rule target,support before,support after,action rule support,confidence before,confidence after,action rule confidence
0,<32.10002;48.06668),S,C,very high,3.0,1.0,female,0.0,1.0,0.059542,0.052672,0.052672,0.604651,0.971831,0.587619
0,<32.10002;48.06668),S,NaN,very high,3.0,1.0,female,10.0,1.0,0.061832,0.051145,0.051145,0.843750,0.626168,0.528329


Based on this table, the decision maker can choose one of the proposed courses of action.

There is one row in the resulting table for each matching action rule. The ID of the action rule is listed in the last column. For example, the first row corresponds to recommendations of Action rule with ID=0. This rule recommends to change the embarkement to Cherbourg and class to the first class. 


Imagine that the ship Titanic II would be created and ready for the first journey. It would copy the cruise of the first Titanic. The shipbuilding company would sell tickets over the Internet. In some cases, the system could recommend different options to increase the probability of survival.


![Recommended actions](img/titanic.png)